In [1]:
import spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import re
import math
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

scope = "playlist-modify-private"

spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

my_id = spotify.current_user()['id']

%run useful_functions.ipynb
%run spotify_scraping.ipynb
%run spotify_scraping_playlists.ipynb

In [2]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()
search = get_spotify_search()
playlists = get_spotify_playlists()
playlist_tracks = get_spotify_playlist_tracks()

master_df = (
broadcast_playlists
.merge(broadcast_details, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.merge(search,left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], how = "left", suffixes=("", " Search"))
.merge(tracks, on='Track ID', how='left', suffixes=("", " Tracks"))
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

del(shows,
    show_broadcast_details,
    broadcast_details,
    broadcast_playlists,
    search)

#set(master_df['Show Name'])

In [3]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Indie Show with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Indie Show with Jack Saunders"
description = "Songs from the Indie Show with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,5NRtdsFFlmyE8qDMgS08PE,2KZedcpHb83f2AXlBDprAV,0
0,08v6GcxD4gF8aZ1yuPry0A,2KZedcpHb83f2AXlBDprAV,1
0,4GvPlSOKfN7aXEuGW8zKUx,2KZedcpHb83f2AXlBDprAV,2
0,4Wsst7bJEVKRWI1vef1S7R,2KZedcpHb83f2AXlBDprAV,3
0,37Q6kTk6hb3nqMCo6KaFln,2KZedcpHb83f2AXlBDprAV,4
...,...,...,...
0,2mdaMDbjFxmxQPO90PxT5w,2KZedcpHb83f2AXlBDprAV,782
0,2JjDl1YbzaBKlhSrUXZqNM,2KZedcpHb83f2AXlBDprAV,783
0,0AIlxnJ1kFpu6wp89rjjUv,2KZedcpHb83f2AXlBDprAV,784
0,7BZWTu4Pv77TxGmmsQaSXN,2KZedcpHb83f2AXlBDprAV,785


In [4]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Sounds with Clara Amfo")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Sounds with Clara Amfo"
description = "Songs from Future Sounds with Clara Amfo in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,61xDJm5YjuIefUvJiC655d,3VTYIgusziIIdXiWB5hB2f,0
0,3i0FkJYlU4MFfYkjFHXXAM,3VTYIgusziIIdXiWB5hB2f,1
0,4W1VDzvrBBG0lZvC221Qh5,3VTYIgusziIIdXiWB5hB2f,2
0,0DmkBOGgEWMbUJlL6VwwF2,3VTYIgusziIIdXiWB5hB2f,3
0,37Q6kTk6hb3nqMCo6KaFln,3VTYIgusziIIdXiWB5hB2f,4
...,...,...,...
0,74RWuKK8uKMbxj4KnJpEgG,3VTYIgusziIIdXiWB5hB2f,586
0,6UoIrO15w3w62Jj8kGCnR7,3VTYIgusziIIdXiWB5hB2f,587
0,5WnvDdcyrFjbIqRxiqlet1,3VTYIgusziIIdXiWB5hB2f,588
0,6JkuJREwd6rd7t1YN069LK,3VTYIgusziIIdXiWB5hB2f,589


In [5]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Artists with Jack Saunders")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Artists with Jack Saunders"
description = "Songs from Future Artists with Jack Saunders in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,4Z3RGx7Be5plLoIw7i83wX,3Wo1IZcPFAfrQJzBpW8QbG,0
0,7Ia03G8qjfSa8fAoCwGRn6,3Wo1IZcPFAfrQJzBpW8QbG,1
0,4bYwK4mvWubnAkCextxH2d,3Wo1IZcPFAfrQJzBpW8QbG,2
0,1jmJHtqll7OoGktgsuENWY,3Wo1IZcPFAfrQJzBpW8QbG,3
0,4GBIZMasDFztSQfK0IrNh6,3Wo1IZcPFAfrQJzBpW8QbG,4
...,...,...,...
0,4IqwX48XrtkoqvZw4hxEfa,3Wo1IZcPFAfrQJzBpW8QbG,952
0,2vT3lchY4bU1BwM69CtDyj,3Wo1IZcPFAfrQJzBpW8QbG,953
0,5BUvQtKdYrWJtdYaSoyFfG,3Wo1IZcPFAfrQJzBpW8QbG,954
0,5cRhrmpBwkZjJ7eFMJKbVM,3Wo1IZcPFAfrQJzBpW8QbG,955


In [6]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Future Alternative")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Future Alternative"
description = "Songs from Future Alternative Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,6X39BP7h0IGcDFnf0PHHOb,0U2xXmHS8QvPrX2WqGT1UH,0
0,1aPZtrvZvmvgu2xIVaPHCo,0U2xXmHS8QvPrX2WqGT1UH,1
0,6H8z76IGQAki6BBvK2xZtQ,0U2xXmHS8QvPrX2WqGT1UH,2
0,1tANZucrXcUr5hInaRu3JC,0U2xXmHS8QvPrX2WqGT1UH,3
0,6ZBr8E9XczoqgMRXyXUe6b,0U2xXmHS8QvPrX2WqGT1UH,4
...,...,...,...
0,5QV6DCj4kvteYBDD9Vt1jY,0U2xXmHS8QvPrX2WqGT1UH,570
0,6L4L6bQPHojlmwBZQLQZjc,0U2xXmHS8QvPrX2WqGT1UH,571
0,5P0iRet3L91fC6SipKKsPl,0U2xXmHS8QvPrX2WqGT1UH,572
0,4ZG0EaIiFpFOaJKmT1ZjNO,0U2xXmHS8QvPrX2WqGT1UH,573


In [7]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Steve Lamacq")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Steve Lamacq"
description = "Songs from Steve Lamacq's Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,5MT6JFRyOTEd1lbkwb0uzD,6Bu0jMJgdN0Emarjx98IL7,0
0,6UN73IYd0hZxLi8wFPMQij,6Bu0jMJgdN0Emarjx98IL7,1
0,6thamYXREbdNCdLQPuIks3,6Bu0jMJgdN0Emarjx98IL7,2
0,6XOHQQJ7RaevE9QjskKxJj,6Bu0jMJgdN0Emarjx98IL7,3
0,5p9dcmTRCkahaqQqzMP6el,6Bu0jMJgdN0Emarjx98IL7,4
...,...,...,...
0,1OO2Fp9PsnaayiekeXuGJX,6Bu0jMJgdN0Emarjx98IL7,1863
0,1ZtE8ysoPZVrMrQ24fVO4f,6Bu0jMJgdN0Emarjx98IL7,1864
0,2AmkpCt3y9NBsB5UgP4S6I,6Bu0jMJgdN0Emarjx98IL7,1865
0,5tmkX2fJ7KUAnMz3W1ibEr,6Bu0jMJgdN0Emarjx98IL7,1866


In [9]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'New Music Fix')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'New Music Fix'
description = "Songs from New Music Fix in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,1IEulXF7S08dapSxx8L0z2,0Cwr3XcLdGZ5ZqDtI7lL11,0
0,4PxmenTJyh7Jd2LX88Y9uv,0Cwr3XcLdGZ5ZqDtI7lL11,1
0,61xDJm5YjuIefUvJiC655d,0Cwr3XcLdGZ5ZqDtI7lL11,2
0,2hvR60sd1TZmrvH89WLatt,0Cwr3XcLdGZ5ZqDtI7lL11,3
0,30kwuy1YdZpxnl28D7UpiO,0Cwr3XcLdGZ5ZqDtI7lL11,4
...,...,...,...
0,1pxLVdC8sjLDA2DF3uOEuW,0Cwr3XcLdGZ5ZqDtI7lL11,889
0,0RTuA2AWVjFw0MURes4kse,0Cwr3XcLdGZ5ZqDtI7lL11,890
0,1MCrtBVXxy2p48uWwnBwEz,0Cwr3XcLdGZ5ZqDtI7lL11,891
0,7xfwMBuatcfbzTSWRfo49f,0Cwr3XcLdGZ5ZqDtI7lL11,892


In [10]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Chillest Show")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Chillest Show"
description = "Songs from the Chillest Show in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,7CFbh9XfJ40XaYWgCbYX3H,1p5mKMwcQVCeQNVkzpUZqJ,0
0,4O3ob7NxMcZLgJ8ucIahPS,1p5mKMwcQVCeQNVkzpUZqJ,1
0,4V7zaJB949FXEfRWj5ADL5,1p5mKMwcQVCeQNVkzpUZqJ,2
0,64I9byMYBlS1ARsC3vtpgW,1p5mKMwcQVCeQNVkzpUZqJ,3
0,1lOtWsnPUdnXn2bcplT02z,1p5mKMwcQVCeQNVkzpUZqJ,4
...,...,...,...
0,6u5lHozM0oGj2l16lTF3KP,1p5mKMwcQVCeQNVkzpUZqJ,1139
0,7oAuqs6akGnPU3Tb00ZmyM,1p5mKMwcQVCeQNVkzpUZqJ,1140
0,38vxstSXciei0IuEO2MoGI,1p5mKMwcQVCeQNVkzpUZqJ,1141
0,2xOdFPQn17smnZrFCtFNSv,1p5mKMwcQVCeQNVkzpUZqJ,1142


In [11]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == "Radio 1's Power Down Playlist with Sian Eleri")].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = "Radio 1's Power Down Playlist with Sian Eleri"
description = "Songs from the Power Down Playlist with Sian Eleri in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,3onYsieuDICdVt5ULZSExF,3Tqe2k7TlEB5OatSMKeppu,0
0,6dVOPXDzlTLyP9slBJ6fhh,3Tqe2k7TlEB5OatSMKeppu,1
0,6RQ5IwG7uADz9LDWliJGjU,3Tqe2k7TlEB5OatSMKeppu,2
0,7rWkd70fyKUGw00s6BBSGz,3Tqe2k7TlEB5OatSMKeppu,3
0,5NtmfJwkA0AXsp2WWHviPn,3Tqe2k7TlEB5OatSMKeppu,4
...,...,...,...
0,1TLHZcMQRMgW3N7hjh9Zlu,3Tqe2k7TlEB5OatSMKeppu,740
0,0tNuJpBgtE65diL6Q8Q7fI,3Tqe2k7TlEB5OatSMKeppu,741
0,3Z3wyUUPcKyw4nJkW2PqBA,3Tqe2k7TlEB5OatSMKeppu,742
0,3INuZoAMZsz0BOiEFTLmAu,3Tqe2k7TlEB5OatSMKeppu,743


In [12]:
demo_df = master_df[(master_df['Service Date'].apply(lambda date: date.year).isin([2022,2023])) & (master_df['Show Name'] == 'BBC Introducing on Radio 1')].drop_duplicates(subset=['Track ID'], keep='first')
new_tracks = list(demo_df['Track ID'].dropna())
title = 'BBC Introducing on Radio 1'
description = "Songs from BBC Introducing on Radio 1 in 2022"
edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

,Track ID,Playlist ID,Position Number
0,28XJPXfzBRT1PxHxHkuB8b,07B5mpuLVdVpgSycsqn4Qc,0
0,16Tzcvd0cxifZtjZplo3nd,07B5mpuLVdVpgSycsqn4Qc,1
0,4HWbqrZUwcHpN9hlLZy9PS,07B5mpuLVdVpgSycsqn4Qc,2
0,7izjWViB7fRzSXuKGoYc6C,07B5mpuLVdVpgSycsqn4Qc,3
0,53NSM8vvR8n0lILTJJTNtA,07B5mpuLVdVpgSycsqn4Qc,4
...,...,...,...
0,2kC9Ccwk1ILSzCN3IBROOx,07B5mpuLVdVpgSycsqn4Qc,724
0,4f5SAiUVc4WFvuxva3JhnR,07B5mpuLVdVpgSycsqn4Qc,725
0,3BOiQtFYI4EE27CyNwm9rU,07B5mpuLVdVpgSycsqn4Qc,726
0,62UTBl5LVd6HOCFvT57Axt,07B5mpuLVdVpgSycsqn4Qc,727


In [13]:
#make_playlist(title, description)
#edit_playlist_tracks(playlists[playlists['name'] == title]['id'][0], new_tracks)

#edit_playlist_details(user=my_id, playlist_id = playlist_id, name = name', description = 'description')
#get_playlist(playlist_id, with_tracks = False)
#get_playlist(playlist_id = playlist_id)
#delete_playlist(playlist_id = playlist_id)

In [14]:
playlist_tracks.to_pickle('../data/spotify_playlist_tracks.pkl')
playlists.to_pickle('../data/spotify_playlists.pkl')
tracks.to_pickle('../data/spotify_tracks.pkl')
albums.to_pickle('../data/spotify_albums.pkl')
artists.to_pickle('../data/spotify_artists.pkl')